1. *TCP-Client*

In [9]:
import socket

target_host = "www.google.com"
target_port = 80

# create a socket object
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# connect the client
client.connect((target_host, target_port))

# send some data
request = b"GET / HTTP/1.1\r\nHost: google.com\r\n\r\n"
client.send(request)

# receive some data
response = client.recv(4096)
print(response.decode())

HTTP/1.1 301 Moved Permanently
Location: http://www.google.com/
Content-Type: text/html; charset=UTF-8
Content-Security-Policy-Report-Only: object-src 'none';base-uri 'self';script-src 'nonce-hH8xol76XlkhNNOiHscHIg' 'strict-dynamic' 'report-sample' 'unsafe-eval' 'unsafe-inline' https: http:;report-uri https://csp.withgoogle.com/csp/gws/other-hp
Date: Sat, 03 Jun 2023 17:21:33 GMT
Expires: Mon, 03 Jul 2023 17:21:33 GMT
Cache-Control: public, max-age=2592000
Server: gws
Content-Length: 219
X-XSS-Protection: 0
X-Frame-Options: SAMEORIGIN

<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">
<TITLE>301 Moved</TITLE></HEAD><BODY>
<H1>301 Moved</H1>
The document has moved
<A HREF="http://www.google.com/">here</A>.
</BODY></HTML>



2. *UDP-Client*

In [6]:
import socket

target_host = "127.0.0.1"
target_port = 9000

# create a socket object
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

try:
    # try to connect to the target host and port
    client.connect((target_host, target_port))
    print(f"Port {target_port} is open and ready to receive data.")
except ConnectionRefusedError:
    print(f"Port {target_port} is not open or not ready to receive data.")
finally:
    # send some data
    client.sendto(b"AAABBBCCC", (target_host, target_port))

    # receive some data
    data, addr = client.recvfrom(4096)
    
   
    print("Received data:", data)
    
    client.close()


Port 9000 is open and ready to receive data.
Received data: b'\xff\x00\x00\x00\x00\x00\x00\x00\x01\x7f'


3. *TCP-Server*

*Server*

In [ ]:
import socket
import threading

bind_ip = "0.0.0.0"
bind_port = 9012

server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
server.bind((bind_ip, bind_port))
server.listen(5)
print(f"[*] Listening on {bind_ip}:{bind_port}")

# Esta es nuestra función para manejar al cliente
def handle_client(client_socket):
    # Imprimimos lo que el cliente envía
    request = client_socket.recv(1024)
    print(f"[*] Received: {request.decode()}")
    # Enviamos de vuelta un paquete
    client_socket.send(b"ACK!")
    client_socket.close()

while True:
    client, addr = server.accept()
    print(f"[*] Accepted connection from: {addr[0]}:{addr[1]}")
    # Creamos un hilo para manejar los datos entrantes del cliente
    client_handler = threading.Thread(target=handle_client, args=(client,))
    client_handler.start()


*Client*

In [ ]:
import socket

target_ip = "0.0.0.1"  # IP del servidor
target_port = 9012      # Puerto del servidor

# Creamos un objeto de socket para el cliente
client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Conectamos el cliente al servidor
client.connect((target_ip, target_port))

# Enviamos un mensaje al servidor
message = "ABCDEF"
client.send(message.encode())

# Recibimos la respuesta del servidor
response = client.recv(1024)
print(f"[*] Received: {response.decode()}")

# Cerramos la conexión del cliente
client.close()


4.  *Replacing Netcat*

In [16]:
import sys
import socket
import getopt
import threading
import subprocess

# define some global variables
listen = False
command = False
upload = False
execute = ""
target = ""
upload_destination = ""
port = 0


In [10]:
def client_sender(buffer):
    client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        # connect to our target host
        client.connect((target, port))
        if len(buffer):
            client.send(buffer.encode())

        while True:
            # now wait for data back
            recv_len = 1
            response = ""
            while recv_len:
                data = client.recv(4096)
                recv_len = len(data)
                response += data.decode()

                if recv_len < 4096:
                    break

            print(response, end='')

            # wait for more input
            buffer = input("")
            buffer += "\n"

            # send it off
            client.send(buffer.encode())

    except Exception as e:
        print("[*] Exception! Exiting.")
        print(str(e))

        # tear down the connection
        client.close()

def server_loop():
    global target

    # if no target is defined, we listen on all interfaces
    if not len(target):
        target = "0.0.0.0"

    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    server.bind((target, port))
    server.listen(5)

    while True:
        client_socket, addr = server.accept()

        # spin off a thread to handle our new client
        client_thread = threading.Thread(target=client_handler, args=(client_socket,))
        client_thread.start()

def run_command(command):
    # trim the newline
    command = command.rstrip()

    # run the command and get the output back
    try:
        output = subprocess.check_output(command, stderr=subprocess.STDOUT, shell=True)
    except subprocess.CalledProcessError as e:
        output = e.output

    # send the output back to the client
    return output.decode()



In [12]:
def client_handler(client_socket):
    global upload
    global execute
    global command

    # check for upload
    if len(upload_destination):
        # read in all of the bytes and write to our destination
        file_buffer = ""

        # keep reading data until none is available
        while True:
            data = client_socket.recv(1024)

            if not data:
                break
            else:
                file_buffer += data

        # now we take these bytes and try to write them out
        try:
            file_descriptor = open(upload_destination, "wb")
            file_descriptor.write(file_buffer)
            file_descriptor.close()

            # acknowledge that we wrote the file out
            client_socket.send(("Successfully saved file to %s\r\n" % upload_destination).encode())
        except Exception as e:
            client_socket.send(("Failed to save file to %s\r\n" % upload_destination).encode())

    # check for command execution
    if len(execute):
        # run the command
        output = run_command(execute)

        client_socket.send(output.encode())

    # now we go into another loop if a command shell was requested
    if command:
        while True:
            # show a simple prompt
            client_socket.send(b"<BHP:#> ")

            # now we receive until we see a linefeed (enter key)
            cmd_buffer = b""
            while b"\n" not in cmd_buffer:
                cmd_buffer += client_socket.recv(1024)

            # send back the command output
            response = run_command(cmd_buffer)

            # send back the response
            client_socket.send(response.encode())


5. *Building a TCP Proxy*

In [20]:
import sys
import socket
import threading


def proxy_handler(client_socket, remote_host, remote_port, receive_first):
    # connect to the remote host
    remote_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    remote_socket.connect((remote_host, remote_port))

    # receive data from the remote end if necessary
    if receive_first:
        remote_buffer = receive_from(remote_socket)
        hexdump(remote_buffer)
        remote_buffer = response_handler(remote_buffer)

        # if we have data to send to our local client, send it
        if len(remote_buffer):
            print("[<==] Sending %d bytes to localhost." % len(remote_buffer))
            client_socket.send(remote_buffer)

    # now let's loop and read from local, send to remote, send to local
    # rinse, wash, repeat
    while True:
        # read from local host
        local_buffer = receive_from(client_socket)
        if len(local_buffer):
            print("[==>] Received %d bytes from localhost." % len(local_buffer))
            hexdump(local_buffer)
            local_buffer = request_handler(local_buffer)

            # send off the data to the remote host
            remote_socket.send(local_buffer)
            print("[==>] Sent to remote.")

        # receive back the response
        remote_buffer = receive_from(remote_socket)
        if len(remote_buffer):
            print("[<==] Received %d bytes from remote." % len(remote_buffer))
            hexdump(remote_buffer)
            remote_buffer = response_handler(remote_buffer)

            # send the response to the local socket
            client_socket.send(remote_buffer)
            print("[<==] Sent to localhost.")

        # if no more data on either side, close the connections
        if not len(local_buffer) or not len(remote_buffer):
            client_socket.close()
            remote_socket.close()
            print("[*] No more data. Closing connections.")
            break


In [ ]:

def server_loop(local_host, local_port, remote_host, remote_port, receive_first):
    server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    try:
        server.bind((local_host, local_port))
    except Exception as e:
        print("[!!] Failed to listen on %s:%d" % (local_host, local_port))
        print("[!!] Check for other listening sockets or correct permissions.")
        sys.exit(0)
    
    print("[*] Listening on %s:%d" % (local_host, local_port))
    server.listen(5)

    while True:
        client_socket, addr = server.accept()

        # print out the local connection information
        print("[==>] Received incoming connection from %s:%d" % (addr[0], addr[1]))

        # start a thread to talk to the remote host
        proxy_thread = threading.Thread(target=proxy_handler, args=(client_socket, remote_host, remote_port, receive_first))
        proxy_thread.start()

def main():
    # no fancy command-line parsing here
    if len(sys.argv[1:]) != 5:
        print("Usage: ./proxy.py [localhost] [localport] [remotehost] [remoteport] [receive_first]")
        print("Example: ./proxy.py 127.0.0.1 9000 10.12.132.1 9000 True")
        sys.exit(0)

    # setup local listening parameters
    local_host = sys.argv[1]
    local_port = int(sys.argv[2])

    # setup remote target
    remote_host = sys.argv[3]
    remote_port = int(sys.argv[4])

    # this tells our proxy to connect and receive data before sending to the remote host
    receive_first = sys.argv[5]

    if "True" in receive_first:
        receive_first = True
    else:
        receive_first = False

    # now spin up our listening socket
    server_loop(local_host, local_port, remote_host, remote_port, receive_first)

main()




